In [1]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import random
import re
import dateutil
import pandas as pd
import html
from names import common
from lxml import etree
from tarfile import TarFile
from lxml.cssselect import CSSSelector

In [3]:
basedir = "./namenindex/"
srcdirs = ['anhalt_dessau_henriette_amalia_von',
           'archives',
           'bourbon_charlotte_de',
           'braunschweig-wolfenbüttel_sophia_hedwig_von',
           'de_witt',
           'elbing',
           'gachard', 
           'grotius',
           'hanover_anne_of',
           'heinsius',
           'huygens_constantijn',
           'oldenbarnevelt',
           'oranje_nassau_albertine_agnes_van',
           'raa',  
           'schutte', 
           'solms_braunfels_amalia_von', 
           'stuart_mary_ii',
           'stuart_mary_i',
           'willemiii', 
           'wvo']

In [4]:
# basedir = "/Users/rikhoekstra/develop/namenindex/"

In [5]:
# os.listdir(basedir)

In [6]:

# srcdirs = ['heinsius', 'schutte', 'raa', 'oldenbarnevelt', 'gachard', 'elbing',
#           'willemiii', 'wvo']
# all_src = ['thorbecke',
#  'wvo',
#  'groen',
#  'beaufort',
#  'zekerheid',
#  'europa',
#  'egodocumenten',
#  'kpp',
#  'willemiii',
#  'nib',
#  'elbing',
#  'staatsregeling',
#  'heinsius',
#  'raa',
#  'archives',
#  'cid',
#  'bwn',
#  'gachard',
#  'oldenbarnevelt',
#  'schutte']

In [7]:
all_src = srcdirs

In [8]:
for f in all_src:
    print ('- ', f)

-  anhalt_dessau_henriette_amalia_von
-  archives
-  bourbon_charlotte_de
-  braunschweig-wolfenbüttel_sophia_hedwig_von
-  de_witt
-  elbing
-  gachard
-  grotius
-  hanover_anne_of
-  heinsius
-  huygens_constantijn
-  oldenbarnevelt
-  oranje_nassau_albertine_agnes_van
-  raa
-  schutte
-  solms_braunfels_amalia_von
-  stuart_mary_ii
-  stuart_mary_i
-  willemiii
-  wvo


In [10]:
# what are we dealing with (these files are biodes)

fn = 'out/0013.xml'

# random sample files, I don't think there is much more information in these files, 
# though there might be in other collections
dr = os.path.join(basedir, srcdirs[3], 'out')
fls = [os.path.join(dr, f) for f in os.listdir(dr) if '.xml' in f]

randomfiles = random.sample(fls, 10)
          
for fl in randomfiles:

    doc = etree.parse(fl)
    root = doc.getroot()
    print (fl)
    for item in root.getchildren():

        print(etree.tostring(item))
    print('\n\n')

FileNotFoundError: [Errno 2] No such file or directory: './namenindex/braunschweig-wolfenbüttel_sophia_hedwig_von/out'

In [8]:
# what are we dealing with (these files are biodes)

fn = 'out/0013.xml'

# random sample files, I don't think there is much more information in these files, 
# though there might be in other collections
dr = os.path.join(basedir, srcdirs[0], 'out')
fls = [os.path.join(dr, f) for f in os.listdir(dr) if '.xml' in f]

randomfiles = random.sample(fls, 10)
          
for fl in randomfiles:

    doc = etree.parse(fl)
    root = doc.getroot()
    print (fl)
    for item in root.getchildren():

        print(etree.tostring(item))
    print('\n\n')

/Users/rikhoekstra/develop/namenindex/heinsius/out/4677.xml
b'<fileDesc>\n    <title/>\n    <ref target="http://www.inghist.nl/retroboeken/heinsius/#accessor=accessor_index&amp;accessor_href=accessor_index%3FSearchSource%253Autf-8%253Austring%3DNoot%252C%2520Roger%2520Walter%2520van%2520der%252C%2520Baron%2520van%2520Carloo"/>\n    <publisher>\n      <name>Instituut voor Nederlandse Geschiedenis</name>\n      <ref target="http://www.inghist.nl/"/>\n    </publisher>\n  </fileDesc>\n  '
b'<person>\n    <persName>Noot, Roger Walter van der, Baron van Carloo</persName>\n  </person>\n  '
b'<biography>\n    <text>(&amp;#134; 1710) Burgemeester van Brussel 1700-02 en lid van de Staten van Brabant.  Wauters, &lt;I&gt;Histoire des environs de Bruxelles&lt;/I&gt;, III, 652.</text>\n  </biography>\n'



/Users/rikhoekstra/develop/namenindex/heinsius/out/2870.xml
b'<fileDesc>\n    <title/>\n    <ref target="http://www.inghist.nl/retroboeken/heinsius/#accessor=accessor_index&amp;accessor_href=acces

In [11]:
# first we need selectors. For these files the following seem to be enough
sel1 = CSSSelector("fileDesc")
sel2 = CSSSelector("ref")
sel3 = CSSSelector("persName")
sel4 = CSSSelector("biography text")

#test some
t = etree.parse("/Users/rikhoekstra/develop/namenindex/heinsius/out/2288.xml")
root = t.getroot()
ref = sel2(root)[0]
url = ref.get("target")
print(url.split('/')[-2:])
s4 = sel4(root)[0]
s4.text

['heinsius', '#accessor=accessor_index&accessor_href=accessor_index%3FSearchSource%253Autf-8%253Austring%3DFrancisco%2520Este%252C%2520Prins%2520van%2520Modena']


'(1698-1780) Oudste zoon van hertog Rinaldo III van Modena, volgt in 1737 zijn  vader op.'

In [12]:
import html
html.unescape("&#134; 1725")

'† 1725'

In [13]:
# hm, upon closer inspection there are quite some life years. Can we extract these?
# who knows if we're also going to do something with the 'zie ook's
# see what we do with the † signs later on

# first make our regular expressions

pat = re.compile('(\(.*[0-9]+.*\))')
byr = re.compile('\(?(?P<gebyr>[0-9]{4})-?(?P<dthyr>[.|†]*|[0-9]{4})\)')

# test on some patterns we can come up with
print('first try')
print(byr.search('(1666-1729)').groupdict()) #expected result {'gebyr': '1666', 'eyr': '1729'}
print(byr.search('(?-1695)').groupdict()) # er {'gebyr': '', 'eyr': '1695'}
print(byr.search('(1695-)').groupdict()) # er {'gebyr': '1695', 'eyr': ''}
print(byr.search('(† 1725)').groupdict()) # er {'gebyr': '', 'eyr': '1725'}
print('\n\n')

t1 = '(1667-1737) Predikant in het Gulikse; 1700-06 te Smyrna; 1707 hoogleraar in de  Oosterse talen aan de Leidse Hogeschool; 1707-09 studiereis naar het Nabije Oosten;  24 maart 1710 aanvaardde hij zijn ambt in Leiden. <I>NNBW</I>.'

print (byr.search(t1).groupdict()) # er {'gebyr': '1667', 'eyr': '1737'}

# nice, but it's the old trap of trying to do everything with one regex. We turn this into a function

def spot_years(text):
    result = {'by': '',
              'dy': ''}
    byr = re.compile('\(([0-9]{4})') 
    eyr = re.compile("([0-9]{4})\)")
    byd = byr.search(text)
    if byd:
        by = byd.groups()[0]
        result['by'] = by
    eyrd = eyr.search(text)
    if eyrd:
        ey = eyrd.groups()[0]
        result['dy'] = ey
    return result

# test on same cases
print('new try with a function')
print(spot_years('(1666-1729)'))
print(spot_years('(?-1695)'))
print(spot_years('(1695-)'))
print(spot_years('(† 1725)'))
print (spot_years(t1)) 


# this seems a lot better

first try
{'gebyr': '1666', 'dthyr': '1729'}
{'gebyr': '1695', 'dthyr': ''}
{'gebyr': '1695', 'dthyr': ''}
{'gebyr': '1725', 'dthyr': ''}



{'gebyr': '1667', 'dthyr': '1737'}
new try with a function
{'by': '1666', 'dy': '1729'}
{'by': '', 'dy': '1695'}
{'by': '1695', 'dy': ''}
{'by': '', 'dy': '1725'}
{'by': '1667', 'dy': '1737'}


In [14]:
inf = etree.parse('/Users/rikhoekstra/develop/namenindex/raa/out/1009.xml')
def get_events(infile):
    """get events such as birthday and death day from biodes"""
    sel = CSSSelector('event')
    sel_p = CSSSelector('place')
    out = {}
    for item in sel(infile):
        t = item.attrib['type'] 
        out[t] = item.attrib['when']
        p = sel_p(item)
        if p is not []:
            p = p[0]
            out["{i}-{p}".format(i=t, p=p.tag)]= p.text

    if 'birth' in out.keys():
        try:
            b = out.get('birth').split('-')[0] # hope birthyears are all same format
            out['by'] = b
        except TypeError:
            pass
        try:
            d = out.get('death').split('-')[0]
            out['dy'] = d
        except TypeError:
            pass
    else:
        t = sel4(infile)
        t = '{}'.format(etree.tostring(t[0]))
        print(t)
        out = spot_years(t)
    return(out)
        

def get_persnaam(infile):
    """get persname from biodes"""
    out = {}
    sel = CSSSelector('name')
    pnames = sel3(infile)[0]
    nms = sel(pnames)
    fullname = " ".join([n.text for n in nms])
    out['fullname'] = fullname
    for item in nms:
        t = item.get('type') 
        out[t] = item.text
    return out 
oefenfile = etree.parse("/Users/rikhoekstra/develop/namenindex/heinsius/out/2288.xml")    
#print(get_events(inf))
print(get_events(oefenfile))

#print(get_persnaam(inf))
print(get_events(oefenfile))

b'<text>(1698-1780) Oudste zoon van hertog Rinaldo III van Modena, volgt in 1737 zijn  vader op.</text>\n  '
{'by': '1698', 'dy': '1780'}
b'<text>(1698-1780) Oudste zoon van hertog Rinaldo III van Modena, volgt in 1737 zijn  vader op.</text>\n  '
{'by': '1698', 'dy': '1780'}


In [15]:
# ok, this does not do what I want, for being from the old namenindex
# we'll create a new indexitem class with the things I need
# it'll take the xml as input and use cssselectors for getting data instead of arcane xpaths
# and steal from the old product what I can use

from names.common import PREFIXES, POSTFIXES, VOORVOEGSELS, TUSSENVOEGSELS, ROMANS
from names.common import  TERRITORIALE_TITELS, STOP_WORDS, R_STOPWORDS #ACHTERVOEGSELS,


class IndexItem(object):
    def __init__(self, src='', content=''):
        """src is the name of an xmlfile with biodes data
        adapted this so that if there is no filename but a string, this will also work"""
        if src != '':
            self.src = etree.parse(src)
        else:
            self.src = etree.fromstring(content)
        #self.names = [] # don't need this
        self.geslachtsnaam = ''
        self.fullname = ''
        self.birth = None
        self.death = None
        self.by = 0 # birth year (or should this be a period with start and end)
        self.dy = 0 # death year
        self.bp = '' # birthplace
        self.dp = '' # dying place
        self.biography = ''
        self.reference = ''
        self.populate()
        
    def populate(self):
        self.get_names()
        self.get_years()
        self.get_biography()
        self.get_reference()
        self.guess_geslachtsnaam()
        
    def get_biography(self):
        b = sel4(self.src)
        if len(b) > 0:
            bio = b[0].text
            bio = html.unescape(bio)
            self.biography = bio
        
    def get_names(self):
        """get names from biodes. adapted this from above"""
        out = {}
        sel = CSSSelector('name')
        pnames = sel3(self.src)[0]
        ptext = pnames.text
        nms = sel(pnames)
        self.fullname = " ".join([n.text for n in nms])
        for item in nms:
            t = item.get('type') 
            setattr(self, t, item.text)
        if self.fullname == '':
            if ptext != '':
                self.fullname = ptext
        
    def get_years(self):
        from IPython.core.debugger import set_trace
        """get events such as birthday and death day from biodes"""
        sel = CSSSelector('event')
        sel_p = CSSSelector('place')
        for item in sel(self.src):
            t = item.attrib['type'] 
            try:
                setattr(self, t, item.attrib['when'])
            except KeyError:
                pass
            p = sel_p(item)
            try:
                if len(p) > 0:
                    p = p[0]
                    plc = "{i}-{p}".format(i=t, p=p.tag)
                    setattr(self, plc, p.text)
            except IndexError:
                print(p)
                set_trace()

        if self.birth or self.death is not None:
            try:
                b = getattr(self, 'birth').split('-')[0] # hope birthyears are all same format
                setattr(self, 'by', b)
            except (TypeError, AttributeError) :
                pass
            try:
                d = getattr(self, 'death').split('-')[0]
                self.dy = d
            except (TypeError, AttributeError):
                pass
        else:
            self.get_yrs_from_txt()
            
    def get_yrs_from_txt(self):
        """try to get birth and death years from biography"""
        t = sel4(self.src)
        try:
            t = '{}'.format(etree.tostring(t[0]))
            out = spot_years(t)
        
            for key in out.keys():
                setattr(self, key, out[key])
        except IndexError:
            pass
        
    def get_reference(self):
        """try to get reference from 'zie verwijzing'"""
        pat = re.compile('.* zie (.*)')
        try:
            reference = pat.search(self.biography).groups()[0]
            reference = reference.strip()
            if len(reference) > 0:
                if reference[-1] == '.':
                    reference = reference[:-1]
            self.reference = reference
        except AttributeError: # lame but useful
            pass

    def _guess_geslachtsnaam_in_string(self, s, hints=[]):
        """TODO strip achtervoegsels (like Junior)"""
        naam = s
        #xXXX dit is HEEL primitief

        #alle woorden die tussen haakjes staan zijn niet de achternaam, en filteren we er uit
        #(maar de haakjes in "Ha(c)ks" blijven staan)
        naam = re.sub(r'(?<!\w)\(.*?\)', '', naam)
        naam = naam.strip()


        if ', ' in naam: #als er een komma in de naam staat, dan is dat wat voor de komma staat de achternaam
            guessed_naam = naam.split(',')[0]
        elif 'startswithgeslachtsnaam' in hints: #er staat geen komma (ofzo) in, maar we weten wel dat de naam
            #met een achternaam begint: dan moet het wel de hele string zijn
            guessed_naam = naam
        elif re.match('[A-Z]\.', naam):
            #als de naam met een initiaal begint, fitleren we alle intiitale er uit, en is de rest de achternaam
            guessed_naam = naam[re.match('([A-Z]\.)+',naam).end():]

        elif ' ' in naam:
            #als er een spatie
            candidates = naam.split(' ')
            if candidates[-1] in ROMANS:
                #als de naam op ene ORMAN numeral eindigt, dan gaan we er van uit dat er geen achternaam is
                #(merk op dat dit mis gaat bij amerikaanse namen als "John Styuivesandt III"
                guessed_naam = ''
            else:
                guessed_naam = candidates[-1]
        else:
            guessed_naam = naam


        #een speciaal geval zijn namen van getrouwde dames, zoals 'Angela Boter-de Groot'
        if '-' in naam:
            for tussenvoegsel in TUSSENVOEGSELS:
                if '-%s' % tussenvoegsel in naam:
                    i = naam.find('-%s' % tussenvoegsel)
                    if i > -1:
                        guessed_naam = self._guess_geslachtsnaam_in_string(naam[:i], hints)
                        guessed_naam = guessed_naam + naam[i:]
        guessed_naam = self._strip_tussenvoegels(guessed_naam)
        return guessed_naam

    def _strip_tussenvoegels(self,s):
        s = s.strip()
        for tussenvoegsel in TUSSENVOEGSELS:
            if s.startswith(tussenvoegsel +  ' ' ):
                s = s[len(tussenvoegsel):]
                s = self._strip_tussenvoegels(s)
                break
        return s.strip()
    
    def guess_geslachtsnaam(self, hints=[], change_xml=True):
        """probeer te raden wat de geslachtsnaam is
        en verander de gegevens xml boom naargelang

        bv:
        >>> naam.to_string()
        '<persName>Puk, Pietje</persName>
        >>> naam.guess_geslachtsnaam()
        >>> naam.to_string()
        '<persName><name type="geslachtsnaam">Puk</name>, Pietje</persName>

        hints: is een lijst met de volgende mogelijkheden:
             ['startswithgeslachtsnaam']

        >>> naam.fromstring('AAA BBB')
        >>> naam.guess_geslachtsnaam()
        'BBB'
        >>> naam.guess_geslachtsnaam(hints=['startswithgeslachtsnaam'])
        'AAA'
        """
        self._root = etree.Element
        if self.geslachtsnaam: #als we al een geslachtsnaam hebben, is er weinig te raden
            pass

        else:
            guessed_geslachtsnaam = self._guess_geslachtsnaam_in_string(self.fullname, hints)
            self.geslachtsnaam = guessed_geslachtsnaam #but guessed
        return self



# test
i = IndexItem('/Users/rikhoekstra/develop/namenindex/raa/out/6294.xml')
r = {k:vars(i)[k] for k in vars(i) if k not in ['src']}
print(r)

i = IndexItem('/Users/rikhoekstra/develop/namenindex/raa/out/3187.xml')
r = {k:vars(i)[k] for k in vars(i) if k not in ['src']}
print(r)

i = IndexItem("/Users/rikhoekstra/develop/namenindex/heinsius/out/2288.xml")
r = {k:vars(i)[k] for k in vars(i) if k not in ['src']}
print(r)


for f in randomfiles:
    i = IndexItem(f)
    r = {k:vars(i)[k] for k in vars(i) if k not in ['src']}
    print(r)

{'geslachtsnaam': 'Merwede', 'fullname': 'mr. Daniel IV van de Merwede', 'birth': None, 'death': '1331', 'by': 0, 'dy': '1331', 'bp': '', 'dp': '', 'biography': '', 'reference': '', 'prepositie': 'mr.', 'voornaam': 'Daniel IV', 'intrapositie': 'van de', 'birth-place': None, 'death-place': None, '_root': <cyfunction Element at 0x1196efba0>}
{'geslachtsnaam': 'Tonckens', 'fullname': 'Joachimus Lunsingh Tonckens', 'birth': None, 'death': '1821', 'by': 0, 'dy': '1821', 'bp': '', 'dp': '', 'biography': '', 'reference': '', 'voornaam': 'Joachimus Lunsingh', 'birth-place': None, 'death-place': None, '_root': <cyfunction Element at 0x1196efba0>}
{'geslachtsnaam': 'Francisco Este', 'fullname': 'Francisco Este, Prins van Modena', 'birth': None, 'death': None, 'by': '1698', 'dy': '1780', 'bp': '', 'dp': '', 'biography': '(1698-1780) Oudste zoon van hertog Rinaldo III van Modena, volgt in 1737 zijn  vader op.', 'reference': '', '_root': <cyfunction Element at 0x1196efba0>}


NameError: name 'randomfiles' is not defined

In [16]:
# we can do something with the following code, but don't know yet if extended item is a good idea
#ok, incorporated partially in indexitem, leave this for further reference

from names.common import PREFIXES, POSTFIXES, VOORVOEGSELS, TUSSENVOEGSELS, ROMANS
from names.common import  TERRITORIALE_TITELS, STOP_WORDS, R_STOPWORDS #ACHTERVOEGSELS,


class ExtendedIndexItem(IndexItem):
    def _guess_geslachtsnaam_in_string(self, s, hints=[]):
            naam = s
            #xXXX dit is HEEL primitief

            #alle woorden die tussen haakjes staan zijn niet de achternaam, en filteren we er uit
            #(maar de haakjes in "Ha(c)ks" blijven staan)
            naam = re.sub(r'(?<!\w)\(.*?\)', '', naam)
            naam = naam.strip()


            if ', ' in naam: #als er een komma in de naam staat, dan is dat wat voor de komma staat de achternaam
                guessed_naam = naam.split(',')[0]
            elif 'startswithgeslachtsnaam' in hints: #er staat geen komma (ofzo) in, maar we weten wel dat de naam
                #met een achternaam begint: dan moet het wel de hele string zijn
                guessed_naam = naam
            elif re.match('[A-Z]\.', naam):
                #als de naam met een initiaal begint, fitleren we alle intiitale er uit, en is de rest de achternaam
                guessed_naam = naam[re.match('([A-Z]\.)+',naam).end():]

            elif ' ' in naam:
                #als er een spatie
                candidates = naam.split(' ')
                if candidates[-1] in ROMANS:
                    #als de naam op ene ORMAN numeral eindigt, dan gaan we er van uit dat er geen achternaam is
                    #(merk op dat dit mis gaat bij amerikaanse namen als "John Styuivesandt III"
                    guessed_naam = ''
                else:
                    guessed_naam = candidates[-1]
            else:
                guessed_naam = naam


            #een speciaal geval zijn namen van getrouwde dames, zoals 'Angela Boter-de Groot'
            if '-' in naam:
                for tussenvoegsel in TUSSENVOEGSELS:
                    if '-%s' % tussenvoegsel in naam:
                        i = naam.find('-%s' % tussenvoegsel)
                        if i > -1:
                            guessed_naam = self._guess_geslachtsnaam_in_string(naam[:i], hints)
                            guessed_naam = guessed_naam + naam[i:]
            guessed_naam = self._strip_tussenvoegels(guessed_naam)
            return guessed_naam

    def _strip_tussenvoegels(self,s):
        s = s.strip()
        for tussenvoegsel in TUSSENVOEGSELS:
            if s.startswith(tussenvoegsel +  ' ' ):
                s = s[len(tussenvoegsel):]
                s = self._strip_tussenvoegels(s)
                break
        return s.strip()
    
    def guess_geslachtsnaam(self, hints=[], change_xml=True):
        """probeer te raden wat de geslachtsnaam is
        en verander de gegevens xml boom naargelang

        bv:
        >>> naam.to_string()
        '<persName>Puk, Pietje</persName>
        >>> naam.guess_geslachtsnaam()
        >>> naam.to_string()
        '<persName><name type="geslachtsnaam">Puk</name>, Pietje</persName>

        hints: is een lijst met de volgende mogelijkheden:
             ['startswithgeslachtsnaam']

        >>> naam.fromstring('AAA BBB')
        >>> naam.guess_geslachtsnaam()
        'BBB'
        >>> naam.guess_geslachtsnaam(hints=['startswithgeslachtsnaam'])
        'AAA'
        """
        self._root = etree.Element
        if self.geslachtsnaam: #als we al een geslachtsnaam hebben, is er weinig te raden
            pass

        else:
            guessed_geslachtsnaam = self._guess_geslachtsnaam_in_string(self.fullname, hints)
            self.geslachtsnaam = guessed_geslachtsnaam #but guessed
        return self


    def guess_normal_form(self, change_xml=True, ):
        """return 'normal form' vna de vorm "Gesalchtsnaam, prepositie voornaam intrapostie, postpostiie"""
        #als we de achternaam niet expliciet is gegeven, dan doen we een educted guess
        try:
            self._root
        except AttributeError:
            self.from_string(self.xml)
        try:
            return self._normal_form
        except AttributeError:

            self.guess_geslachtsnaam(change_xml=change_xml)
            if self.geslachtsnaam():
                rest = self.serialize(exclude=['geslachtsnaam'])
                rest = rest.replace(',', '')
                rest = rest.replace('  ', ' ')
                rest = rest.strip()
                if rest:
                    s = '%s, %s' % (self.geslachtsnaam(),  rest)
                else:
                    s = '%s' % (self.geslachtsnaam())
            else:
                s = self.serialize()

            #alles wat tussen haakjes staat gaat er uit
            s = re.sub('\(.*?\)', '', s)

            #alles wat in HOOFDLETTERS staat wordt Hoofdletters, behalve tussenvoegsels en romans
            result = ''
            for s in s.split():
                if s == s.lower():
                    pass
                elif s in ROMANS + TUSSENVOEGSELS:
                    pass
                elif u'.' in s:
                    pass
                else:
                    s =  s.capitalize()
                result += ' '  + s
            result = result.strip()
            self._normal_form = result
        return result


    def initials(self):

        s = self.guess_normal_form()
        return ' '.join([s[0] for s in re.findall('\w+', s) if s not in STOP_WORDS])
    
    def _name_parts(self):
        s = self.serialize()
        return re.findall('\S+', s)

    def contains_initials(self):
        #all parts of the name are initials, except  "geslachtsnaam" or ROMANS or TUSSENVOEGSELS
        g = self.guess_geslachtsnaam().geslachtsnaam()
        for p in self._name_parts():
            if p.endswith('.') and p not in VOORVOEGSELS + TERRITORIALE_TITELS:
                return True
#            if (p in g) or p.endswith('.') or p.endswith(',') or p in TUSSENVOEGSELS:
#                pass
#           else:
#               return False
        return False

In [17]:
tar = TarFile.open(os.path.join(basedir, srcdirs[0], "out.tar.gz"), "r:gz")
rescoll = []
record = {}
for member in tar.getmembers():
    record = {}
    by = ''
    ey = ''
    f = tar.extractfile(member)
    if f is not None:
        content = f.read()
    doc = etree.fromstring(content)
    root = doc.getroottree()
    name = sel3(root)[0]
    name = name.text
    ref = sel2(root)[0]
    url = ref.get("target")
    collection = url.split('/')[-2:]
    bio = sel4(root)
    if bio:
        bio = bio[0].text
        bio = html.unescape(bio)
        result = pat.search(bio)    
        if result:
            try:
                r = spot_years(result.groups()[0])
                by = r.get('gebyr')
                ey= r.get('eyr')
            except AttributeError:
                pass
    record['name'] = name
    record['collection'] = collection[0]
    record['path'] = collection[1]
    record['bio'] = bio
    record['birthyear'] = by
    record['deceased'] = ey
    rescoll.append(record)

df = pd.DataFrame().from_records(rescoll)
df.head(30)



FileNotFoundError: [Errno 2] No such file or directory: './namenindex/anhalt_dessau_henriette_amalia_von/out.tar.gz'

In [18]:
baseurl = "http://resources.huygens.knaw.nl/retroboeken/heinsius/"
toturl = baseurl + df.iloc[0].path
toturl

# hmm, index is offline

NameError: name 'df' is not defined

In [19]:
#maar zo kan het ook
baseurl = "http://resources.huygens.knaw.nl/retro/search/?SearchableText%3Austring%3Autf8=Deynaud" # daar gaan we nog s op kauwen

In [20]:
df.to_json('heinsiusindex.json', orient="records")

NameError: name 'df' is not defined

Does this also work with another source? I tried it and it does, so we're going to generalize it

_Edit:_ this was a bit too fast; it worked by no means with all collections 

In [19]:
def convert_source(basedir, srcdir, basurl=baseurl):
    tar = TarFile.open(os.path.join(basedir, srcdir, "out.tar.gz"), "r:gz")
    rescoll = []
    record = {}
    for member in tar.getmembers():
        record = {}
        by = ''
        ey = ''
        f = tar.extractfile(member)
        if f is not None:
            content = f.read()
        doc = etree.fromstring(content)
        root = doc.getroottree()
        name = sel3(root)[0]
        name = name.text
        ref = sel2(root)[0]
        url = ref.get("target")
        collection = url.split('/')[-2:]
        bio = sel4(root)
        if bio:
            bio = bio[0].text
            bio = html.unescape(bio)
            result = pat.search(bio)    
            if result:
                try:
                    r = spot_years(result.groups()[0])
                    by = r.get('gebyr')
                    ey= r.get('eyr')
                except AttributeError:
                    pass
        record['name'] = name
        record['collection'] = collection[0]
        record['path'] = collection[1]
        record['bio'] = bio
        record['birthyear'] = by
        record['deceased'] = ey
        rescoll.append(record)

    df2 = pd.DataFrame().from_records(rescoll)
    return df2



In [20]:
from tqdm import tqdm_notebook
def convert_sources(basedir, srcdir):
    tar = TarFile.open(os.path.join(basedir, srcdir, "out.tar.gz"), "r:gz")
    rescoll = []
    members = tar.getmembers()
    progressbar1 = tqdm_notebook(total=len(members))
    for member in members:
        f = tar.extractfile(member)
        if f is not None:
            content = f.read()
        i = ExtendedIndexItem(content=content)
        r = {k:vars(i)[k] for k in vars(i) if k not in ['src', '_root']}
        r['collection'] = srcdir
        rescoll.append(r)
        progressbar1.update()
    df = pd.DataFrame().from_records(rescoll)
    return df

In [21]:

dfs = ''
progressbar1 = tqdm_notebook(total=len(srcdirs))
for item in srcdirs:
    print("converting: ", item)
    progressbar1.update()
    if not type(dfs) == type(pd.DataFrame()):
        dfs = convert_sources(basedir, item)
    else:
        print("appending: ", item)
        dfss = convert_sources(basedir, item)
        dfs = pd.concat([dfs, dfss], ignore_index=True, sort=False)
dfs


/var/folders/9y/s7573g1n3q9fz78c71ryxj4m0000gn/T/ipykernel_89366/3284577240.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progressbar1 = tqdm_notebook(total=len(srcdirs))


  0%|          | 0/8 [00:00<?, ?it/s]

converting:  heinsius


/var/folders/9y/s7573g1n3q9fz78c71ryxj4m0000gn/T/ipykernel_89366/834589833.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progressbar1 = tqdm_notebook(total=len(members))


  0%|          | 0/6899 [00:00<?, ?it/s]

...

  0%|          | 0/511 [00:00<?, ?it/s]

converting:  wvo
appending:  wvo


/var/folders/9y/s7573g1n3q9fz78c71ryxj4m0000gn/T/ipykernel_89366/834589833.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progressbar1 = tqdm_notebook(total=len(members))


  0%|          | 0/3078 [00:00<?, ?it/s]

,geslachtsnaam,fullname,birth,death,by,dy,bp,dp,biography,reference,collection,voornaam,intrapositie,birth-place,death-place,prepositie
0,Deynaud,"Deynaud, N",None,None,,,,,"N. Hugenoots predikant in Duitsland, met een ...",,heinsius,NaN,NaN,NaN,NaN,NaN
1,La Tour,"La Tour, François-Egon de, prins van Auvergne,...",None,None,,,,,"La Tour, François-Egon de, prins van Auvergne,...",,heinsius,NaN,NaN,NaN,NaN,NaN
2,Grignan,"Grignan, François d'Adhémar de Monteil, Graaf van",None,None,,1715,,,(† 1715) Frans edelman en offcier; luitenant-g...,,heinsius,NaN,NaN,NaN,NaN,NaN
3,Watrigand,"Watrigand, Jacques",None,None,,1708,,,(† 1708) Nederlands officier; ritmeester in he...,,heinsius,NaN,NaN,NaN,NaN,NaN
4,Aa,"Aa, Boudewijn van der",None,None,,,,,Broer van de Leidse boekdrukker Pieter van der...,,heinsius,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64105,Witte,Klaas de Witte,None,None,0,0,,,,,wvo,NaN,NaN,NaN,NaN,NaN
64106,Guillaume de Maulde,"Guillaume de Maulde, heer van Mansart",None,None,0,0,,,,,wvo,NaN,NaN,NaN,NaN,NaN
64107,Lostadt,Elsabe van Lostadt,None,None,0,0,,,,,wvo,NaN,NaN,NaN,NaN,NaN
64108,Baert,Hans Baert,None,None,0,0,,,,,wvo,NaN,NaN,NaN,NaN,NaN


In [24]:
dfs.to_csv('/Users/rikhoekstra/Downloads/namenindex_20231116.tsv.gz',sep='\t',compression='gzip')

In [27]:
dfs.collection.describe()
dfs.loc[dfs.collection == 'raa']

,geslachtsnaam,fullname,birth,death,by,dy,bp,dp,biography,reference,collection,voornaam,intrapositie,birth-place,death-place,prepositie
18837,Haan,Pieter Josephus de Haan,None,1828,0,1828,,,,,raa,Pieter Josephus,de,None,None,NaN
18838,Valencijn,Martinus comte (Empire) Valencijn,None,1785-02-14,0,1785,,,,,raa,Martinus,comte (Empire),None,None,NaN
18839,Reede,mr.dr. Godert Adriaan hertog van Reede,None,1691-09-09,0,1691,,,,,raa,Godert Adriaan,hertog van,None,None,mr.dr.
18840,Meulen,mr.dr. Quirijn comte (Empire) van der Meulen,None,None,0,0,,,,,raa,Quirijn,comte (Empire) van der,None,None,mr.dr.
18841,Lantman,Jacob Pietersz. comte (Empire) Lantman,None,None,0,0,,,,,raa,Jacob Pietersz.,comte (Empire),None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58838,Heeren,Walterus Johannis Heeren,None,1838,0,1838,,,,,raa,Walterus Johannis,NaN,None,None,NaN
58839,Cappelle,Aart van Cappelle,None,None,0,0,,,,,raa,Aart,van,None,None,NaN
58840,graaf,mr. Klaas graaf,None,None,0,0,,,,,raa,Klaas,graaf,None,None,mr.
58841,Groos,dr. Nicolaas Sikkel Groos,2004,None,2004,0,,,,,raa,Nicolaas,Sikkel,Woerden,None,dr.


In [28]:

ndfs = ''
progressbar1 = tqdm_notebook(total=len(srcdirs))
for item in all_src:
    print("converting: ", item)
    progressbar1.update()
    if not type(dfs) == type(pd.DataFrame()):
        ndfs = convert_sources(basedir, item)
    else:
        print("appending: ", item)
        dfss = convert_sources(basedir, item)
        ndfs = pd.concat([dfs, dfss], ignore_index=True, sort=False)
ndfs


/var/folders/9y/s7573g1n3q9fz78c71ryxj4m0000gn/T/ipykernel_44182/2492982288.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progressbar1 = tqdm_notebook(total=len(srcdirs))


  0%|          | 0/8 [00:00<?, ?it/s]

/var/folders/9y/s7573g1n3q9fz78c71ryxj4m0000gn/T/ipykernel_44182/834589833.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progressbar1 = tqdm_notebook(total=len(members))


converting:  thorbecke
appending:  thorbecke


  0%|          | 0/2002 [00:00<?, ?it/s]

...

  0%|          | 0/453 [00:00<?, ?it/s]

converting:  schutte
appending:  schutte


/var/folders/9y/s7573g1n3q9fz78c71ryxj4m0000gn/T/ipykernel_44182/834589833.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progressbar1 = tqdm_notebook(total=len(members))


  0%|          | 0/11938 [00:00<?, ?it/s]

,geslachtsnaam,fullname,birth,death,by,dy,bp,dp,biography,reference,collection,voornaam,intrapositie,birth-place,death-place,prepositie
0,Deynaud,"Deynaud, N",None,None,,,,,"N. Hugenoots predikant in Duitsland, met een ...",,heinsius,NaN,NaN,NaN,NaN,NaN
1,La Tour,"La Tour, François-Egon de, prins van Auvergne,...",None,None,,,,,"La Tour, François-Egon de, prins van Auvergne,...",,heinsius,NaN,NaN,NaN,NaN,NaN
2,Grignan,"Grignan, François d'Adhémar de Monteil, Graaf van",None,None,,1715,,,(† 1715) Frans edelman en offcier; luitenant-g...,,heinsius,NaN,NaN,NaN,NaN,NaN
3,Watrigand,"Watrigand, Jacques",None,None,,1708,,,(† 1708) Nederlands officier; ritmeester in he...,,heinsius,NaN,NaN,NaN,NaN,NaN
4,Aa,"Aa, Boudewijn van der",None,None,,,,,Broer van de Leidse boekdrukker Pieter van der...,,heinsius,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76043,Broedelet,"Broedelet, Jacobus",None,None,0,0,,,,,schutte,NaN,NaN,NaN,NaN,NaN
76044,Milan(o),Milan(o),None,None,0,0,,,,,schutte,NaN,NaN,NaN,NaN,NaN
76045,James,"James, James French",None,None,0,0,,,,,schutte,NaN,NaN,NaN,NaN,NaN
76046,Knuyt,"Knuyt, Johan de",None,None,0,0,,,,,schutte,NaN,NaN,NaN,NaN,NaN


In [29]:
ndfs.reset_index(drop=True, inplace=True)

In [28]:
ndfs.to_json('./ni_new_2021.json', orient="records")
len(ndfs)

56045

In [29]:
def convert_sources(basedir, srcdir):
    tar = TarFile.open(os.path.join(basedir, srcdir, "out.tar.gz"), "r:gz")
    rescoll = []
    for member in tar.getmembers():
        f = tar.extractfile(member)
        if f is not None:
            content = f.read()
        i = ExtendedIndexItem(content=content)
        r = {k:vars(i)[k] for k in vars(i) if k not in ['src']}
        r['collection'] = srcdir
        rescoll.append(r)
    df = pd.DataFrame().from_records(rescoll)
    return df

# To Elastic Search

In [48]:
# Run this cell to import the necessary libraries and initiate an Elasticsearch object.
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import RequestError
import datetime

import json
from IPython.core.display import display, HTML
import re

es = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True) 

In [63]:
def init_es(es='', index='', doc_type='', jsonsrc='', jsonfl=''):
    es = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True)
    # create mapping in index (the index should already exist)
    index = index
    indextype = doc_type
    if not es.indices.exists(index):
        es.indices.create(index)
    request_body = {
                "naam": {
                        "properties":{
                            'geslachtsnaam': {"type":"string"},
                            'fullname': {"type":"string"},
                            'birth': {"type":"string"},
                            'death': {"type":"string"},
                            'by': {"type":"integer"},
                            'dy': {"type":"integer"},
                            'bp': {"type":"string"},
                            'dp': {"type":"string"},
                            'biography': {"type":"text"},
                            'reference': {"type":"string"}
                        }
                    },
        }

    INDEX_NAME=index
    es.indices.put_mapping(index = index, doc_type=doc_type,  body=request_body)

    if not jsonsrc:
        data_dict = json.load('ni_nw.json')
    else:
        data_dict = jsonsrc

    bulk_data = []
    for item in data_dict:
        try:
            bulk_data.append({'index':{'_type':indextype,
                                       '_index':index,
                                       '_id':'%s' % item.get('id')}})
            bulk_data.append(item)
        except KeyError:
            print (item)

    es.bulk(index=index, body=bulk_data, refresh=True)

In [ ]:
with open('./ni_nw.json') as fl:
    jsonsrc = json.load(fp=fl)
init_es(es=es,
        index='namen_new',
        doc_type='naam', 
        jsonsrc=jsonsrc, jsonfl=''
       )

In [76]:
data_dict = jsonsrc

In [71]:
data_dict.keys()

dict_keys(['geslachtsnaam', 'fullname', 'birth', 'death', 'by', 'dy', 'bp', 'dp', 'biography', 'reference', 'collection', 'voornaam', 'intrapositie', 'birth-place', 'death-place', 'prepositie'])

In [77]:
index = 'index'
indextype = 'naam'
bulk_data = []
for item in data_dict:
    try:
        bulk_data.append({'index':{'_type':indextype,
                                   '_index':index,
                                   '_id':'%s' % item.get('id')}})
        bulk_data.append(item)
    except KeyError:
        print (item)


In [79]:
bulk_data[:10]

[{'index': {'_type': 'naam', '_index': 'index', '_id': 'None'}},
 {'geslachtsnaam': 'Deynaud',
  'fullname': 'Deynaud, N',
  'birth': None,
  'death': None,
  'by': '',
  'dy': '',
  'bp': '',
  'dp': '',
  'biography': 'N.  Hugenoots predikant in Duitsland, met een jaargeld van de Staten-Generaal, 1710.',
  'reference': '',
  'collection': 'heinsius',
  'voornaam': None,
  'intrapositie': None,
  'birth-place': None,
  'death-place': None,
  'prepositie': None},
 {'index': {'_type': 'naam', '_index': 'index', '_id': 'None'}},
 {'geslachtsnaam': 'La Tour',
  'fullname': 'La Tour, François-Egon de, prins van Auvergne, markies van',
  'birth': None,
  'death': None,
  'by': '',
  'dy': '',
  'bp': '',
  'dp': '',
  'biography': 'La Tour, François-Egon de, prins van Auvergne, markies van',
  'reference': '',
  'collection': 'heinsius',
  'voornaam': None,
  'intrapositie': None,
  'birth-place': None,
  'death-place': None,
  'prepositie': None},
 {'index': {'_type': 'naam', '_index': 'in

In [80]:
data_dict[:10]

[{'geslachtsnaam': 'Deynaud',
  'fullname': 'Deynaud, N',
  'birth': None,
  'death': None,
  'by': '',
  'dy': '',
  'bp': '',
  'dp': '',
  'biography': 'N.  Hugenoots predikant in Duitsland, met een jaargeld van de Staten-Generaal, 1710.',
  'reference': '',
  'collection': 'heinsius',
  'voornaam': None,
  'intrapositie': None,
  'birth-place': None,
  'death-place': None,
  'prepositie': None},
 {'geslachtsnaam': 'La Tour',
  'fullname': 'La Tour, François-Egon de, prins van Auvergne, markies van',
  'birth': None,
  'death': None,
  'by': '',
  'dy': '',
  'bp': '',
  'dp': '',
  'biography': 'La Tour, François-Egon de, prins van Auvergne, markies van',
  'reference': '',
  'collection': 'heinsius',
  'voornaam': None,
  'intrapositie': None,
  'birth-place': None,
  'death-place': None,
  'prepositie': None},
 {'geslachtsnaam': 'Grignan',
  'fullname': "Grignan, François d'Adhémar de Monteil, Graaf van",
  'birth': None,
  'death': None,
  'by': '',
  'dy': '1715',
  'bp': '',
 